In [9]:
import numpy as np
from pydrake.geometry import StartMeshcat
from pydrake.multibody.inverse_kinematics import (
    DifferentialInverseKinematicsParameters,
    DifferentialInverseKinematicsStatus,
    DoDifferentialInverseKinematics,
)
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder, EventStatus, LeafSystem
from pydrake.visualization import MeshcatPoseSliders

from manipulation import running_as_notebook
from manipulation.meshcat_utils import WsgButton
from manipulation.scenarios import AddIiwaDifferentialIK, ExtractBodyPose
from manipulation.station import MakeHardwareStation, load_scenario

In [10]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7006


In [11]:
def construct_scenario():
    scenario_data = """
directives:
- add_model:
    name: table_top
    file: package://tucker/models/table_top.sdf
- add_weld:
    parent: world
    child: table_top::table_top_center
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: table_top_link
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [0.35, 0.5, 0.015]
- add_model:
    name: wsg
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
"""
# plant_config:
#    time_step: 0.01

    driver_data = """
model_drivers:
    iiwa: !IiwaDriver
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {}
"""

    scenario_data += driver_data
    scenario = load_scenario(data = scenario_data)
    return scenario

In [12]:
def teleop_3d():
    meshcat.ResetRenderMode()
    builder = DiagramBuilder()
    scenario = construct_scenario()
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat, package_xmls=['./package.xml']))
    plant = station.GetSubsystemByName("plant")
    #station = builder.AddSystem(MakeHardwareStation(scenario, meshcat))
    # TODO(russt): Replace with station.AddDiffIk(...)
    controller_plant = station.GetSubsystemByName(
        "iiwa.controller"
    ).get_multibody_plant_for_control()
    # Set up differential inverse kinematics.
    differential_ik = AddIiwaDifferentialIK(
        builder,
        controller_plant,
        frame=controller_plant.GetFrameByName("iiwa_link_7"),
    )
    builder.Connect(
        differential_ik.get_output_port(),
        station.GetInputPort("iiwa.position"),
    )
    builder.Connect(
        station.GetOutputPort("iiwa.state_estimated"),
        differential_ik.GetInputPort("robot_state"),
    )

    # Set up teleop widgets.
    meshcat.DeleteAddedControls()
    teleop = builder.AddSystem(
        MeshcatPoseSliders(
            meshcat,
            lower_limit=[0, -0.5, -np.pi, -0.6, -0.8, 0.0],
            upper_limit=[2 * np.pi, np.pi, np.pi, 0.8, 0.3, 1.1],
        )
    )
    builder.Connect(
        teleop.get_output_port(), differential_ik.GetInputPort("X_WE_desired")
    )
    # Note: This is using "Cheat Ports". For it to work on hardware, we would
    # need to construct the initial pose from the HardwareStation outputs.
    plant = station.GetSubsystemByName("plant")
    ee_pose = builder.AddSystem(
        ExtractBodyPose(
            station.GetOutputPort("body_poses"),
            plant.GetBodyByName("iiwa_link_7").index(),
        )
    )
    builder.Connect(
        station.GetOutputPort("body_poses"), ee_pose.get_input_port()
    )
    builder.Connect(ee_pose.get_output_port(), teleop.get_input_port())
    wsg_teleop = builder.AddSystem(WsgButton(meshcat))
    builder.Connect(
        wsg_teleop.get_output_port(0), station.GetInputPort("wsg.position")
    )

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.get_mutable_context()
    simulator_context = simulator.get_mutable_context()
    plant_context = plant.GetMyMutableContextFromRoot(simulator_context)
    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)

        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
            print(simulator.get_context().get_continuous_state_vector())    
        meshcat.DeleteButton("Stop Simulation")

    else:
        simulator.AdvanceTo(0.1)


teleop_3d()

Press Space to open/close the gripper
Press Escape to stop the simulation
[-9.511088270919043e-05, 1.769809010979125e-05, -5.3522004307926086e-05, 1.1947922375460918e-05, 1.8826614563509287e-05, -2.770294530139979e-05, -0.00014113860987291719]
[-0.0001623706343408068, 1.5182480178374349e-05, -4.376691514830712e-06, 1.0095119007633122e-05, 2.1330605721085313e-05, -3.1648184966621845e-05, -0.00015797397098544274]
[0.0027450189581333146, -0.010895301104795412, 0.007447692388386375, 0.03961535260801699, -0.0023128849089006495, 0.2679158532564374, -0.0005255675890528205]
[0.002837408015705828, -0.011210448256136991, 0.00020079135764929098, 0.04181553776391654, 0.012762513100813578, 0.2797240892507199, -0.008940845483175376]
[0.010519717998671501, -0.010932131451932694, -0.013257180287506921, 0.03979894403491012, -0.007622026136037491, 0.280553017961558, -0.010578019852225097]
[0.013931451118258489, 0.007352807057157911, 0.0025320315359814408, 0.04971494252388212, -0.012631222621497453, 0.27

[0.0006461519677933631, -0.007565999108112394, -0.004750259143491282, 0.031994668643530295, 0.015518202146902423, 0.23107354666430446, -0.01616014752447438]


[-0.018086731998394497, -0.04281571355920349, 0.019510869184625846, -0.29697529597838584, -0.012356924432539015, 0.15795199496510628, 0.01191220409424085]
[-0.07174845268314406, -0.09199002654026868, 0.015174667928074368, -0.80848891396763, 0.004330130878576715, 0.03476759675009788, -0.036137437697905686]
